In [1]:
!pip install -r ../requirements.txt
!pip install sklearn
!pip install tensorflow addons

  Using cached AddOns-0.7.zip (34 kB)
    ERROR: Command errored out with exit status 1:
     command: /home/studio-lab-user/.conda/envs/default/bin/python3.9 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-r8usia01/addons_91fad6a269334d61906626a9b0fd41b9/setup.py'"'"'; __file__='"'"'/tmp/pip-install-r8usia01/addons_91fad6a269334d61906626a9b0fd41b9/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-8q6sayj1
         cwd: /tmp/pip-install-r8usia01/addons_91fad6a269334d61906626a9b0fd41b9/
    Complete output (8 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-install-r8usia01/addons_91fad6a269334d61906626a9b0fd41b9/setup.py", line 4, 

In [14]:
import tensorflow as tf
import pandas as pd
from sklearn.metrics import classification_report
from transformers import BertTokenizerFast, TFBertForSequenceClassification, pipeline
from collections import Counter

MAX_LENGTH = 200# Define the maximum number of words to tokenize (BERT can tokenize up to 512)

# Create the model

In [15]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased",from_tf=True,)
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased",)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Load and transform the data

In [16]:
datasets = pd.read_csv('../datasets/train_preprocessed.csv').dropna()
X_train,y_train = datasets["content"].astype("string"),datasets["sentiment"].astype("category").cat.codes

In [17]:
# Define function to encode text data in batches
def batch_encode(tokenizer, texts, batch_size=256, max_length=MAX_LENGTH):
    """""""""
    A function that encodes a batch of texts and returns the texts'
    corresponding encodings and attention masks that are ready to be fed 
    into a pre-trained transformer model.
    
    Input:
        - tokenizer:   Tokenizer object from the PreTrainedTokenizer Class
        - texts:       List of strings where each string represents a text
        - batch_size:  Integer controlling number of texts in a batch
        - max_length:  Integer controlling max number of words to tokenize in a given text
    Output:
        - input_ids:       sequence of texts encoded as a tf.Tensor object
        - attention_mask:  the texts' attention mask encoded as a tf.Tensor object
    """""""""
    
    input_ids = []
    attention_mask = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             max_length=max_length,
                                             padding='max_length', #implements dynamic padding
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_token_type_ids=False
                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])
    
    
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)
    
    
# Encode X_train
X_train_ids, X_train_attention = batch_encode(tokenizer, X_train.tolist())

Define F1 Loss

In [18]:
def macro_double_soft_f1(y, y_hat):
    """Compute the macro soft F1-score as a cost (average 1 - soft-F1 across all labels).
    Use probability values instead of binary predictions.
    This version uses the computation of soft-F1 for both positive and negative class for each label.
    
    Args:
        y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        
    Returns:
        cost (scalar Tensor): value of the cost function for the batch
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    tn = tf.reduce_sum((1 - y_hat) * (1 - y), axis=0)
    soft_f1_class1 = 2*tp / (2*tp + fn + fp + 1e-16)
    soft_f1_class0 = 2*tn / (2*tn + fn + fp + 1e-16)
    cost_class1 = 1 - soft_f1_class1 # reduce 1 - soft-f1_class1 in order to increase soft-f1 on class 1
    cost_class0 = 1 - soft_f1_class0 # reduce 1 - soft-f1_class0 in order to increase soft-f1 on class 0
    cost = 0.5 * (cost_class1 + cost_class0) # take into account both class 1 and class 0
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost

# Create and compile the model

In [19]:
input_ids_layer = tf.keras.layers.Input(shape=(200,), 
                                        name='input_ids', 
                                        dtype='int32')
input_attention_layer = tf.keras.layers.Input(shape=(200,), 
                                              name='input_attention', 
                                              dtype='int32')
last_hidden_state = model.bert([input_ids_layer, input_attention_layer])[0]

one_more = tf.keras.layers.AveragePooling1D(1,200)(last_hidden_state)
pooling = tf.keras.layers.Reshape((768,))(one_more)
output = tf.keras.layers.Dense(13,activation='softmax')(pooling)
roberta = tf.keras.Model([input_ids_layer, input_attention_layer], output)

In [20]:
params = {'optimizer':tf.keras.optimizers.Adam(learning_rate=1e-4),
          'loss':tf.keras.losses.CategoricalCrossentropy(),
          'metrics':["accuracy"],# tf.keras.metrics.top_k_categorical_accuracy(k=3)
}
roberta.compile(**params)

In [21]:
roberta.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 200)]        0           []                               
                                                                                                  
 input_attention (InputLayer)   [(None, 200)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'input_attention[0][0]']        
                                tentions(last_hidde                                               
                                n_state=(None, 200,                                         

# Created weights for each class and Train

In [22]:
#counts = Counter(y_train)
#weights = {i:1/j for i,j in counts.items()}

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath="./esign_checkpoint/",save_best_only=True,save_weights_only=False),
]

roberta.fit(x=[X_train_ids,X_train_attention],
            y=pd.get_dummies(y_train),
            batch_size=8,
            epochs = 2,
            workers = -1,
           #validation_split=0.2,
            callbacks = callbacks
            #class_weight = weights
           )

Epoch 1/2
1541/3343 [============>.................] - ETA: 3:03:01 - loss: 2.1980 - accuracy: 0.2132

In [11]:
 model.save_weights('./saved_weights.h5')

In [ ]:
tf.keras.models.save_model(roberta,'./full_models/augmented')

In [12]:
from sklearn.metrics import classification_report
from tensorflow import keras
import tqdm

#load the test set
datasets = pd.read_csv('../datasets/test_preprocessed.csv').dropna()# recreate for non-augmented
X_test,y_test = datasets["content"].astype("string"),datasets["sentiment"].astype("category").cat.codes

X_test_ids, X_test_attention = batch_encode(tokenizer, X_test.tolist())

TypeError: in user code:

    File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/keras/engine/training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/keras/engine/training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/keras/engine/training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/keras/engine/training.py", line 1751, in predict_step
        return self(x, training=False)
    File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None

    TypeError: Exception encounte

In [10]:
tqdm_callback = tfa.callbacks.TQDMProgressBar()

In [11]:
# normal EmoRoBERTa
roberta.load_weights('weights/without_class_weights/saved_weights.h5')
y_pred = tf.argmax(roberta.predict([X_test_ids, X_test_attention],workers = -1,callbacks=[tqdm_callback],verbose=1),axis = 1)
print(classification_report(y_test, y_pred,zero_division = 0))

461/461 [==============================] - 183s 391ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        73
           1       0.00      0.00      0.00       118
           2       0.00      0.00      0.00       538
           3       0.00      0.00      0.00       501
           4       0.00      0.00      0.00       586
           5       0.37      0.47      0.41      1719
           6       0.67      0.05      0.10       873
           7       0.58      0.43      0.49      1268
           8       0.36      0.67      0.47      2825
           9       0.81      0.04      0.08      1007
          10       0.37      0.36      0.37      1704
          11       0.39      0.04      0.08       722
          12       0.32      0.51      0.39      2790

    accuracy                           0.37     14724
   macro avg       0.30      0.20      0.18     14724
weighted avg       0.38      0.37      0.31     14724



In [12]:
# weighted EmoRoBERTa
roberta.load_weights('weights/with_class_weights/saved_weights.h5')
y_pred = tf.argmax(roberta.predict([X_test_ids, X_test_attention],workers = -1,callbacks=[tqdm_callback],verbose=1),axis = 1)
print(classification_report(y_test, y_pred,zero_division = 0))

461/461 [==============================] - 190s 412ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        73
           1       0.04      0.37      0.07       118
           2       0.00      0.00      0.00       538
           3       0.10      0.01      0.01       501
           4       0.20      0.12      0.15       586
           5       0.38      0.11      0.18      1719
           6       0.22      0.63      0.32       873
           7       0.61      0.26      0.37      1268
           8       0.37      0.34      0.35      2825
           9       0.13      0.58      0.21      1007
          10       0.30      0.32      0.31      1704
          11       0.22      0.12      0.16       722
          12       0.48      0.04      0.07      2790

    accuracy                           0.24     14724
   macro avg       0.23      0.22      0.17     14724
weighted avg       0.34      0.24      0.22     14724



In [13]:
# f1 loss EmoRoBERTa
roberta.load_weights('weights/f1_loss/saved_weights.h5')
y_pred = tf.argmax(roberta.predict([X_test_ids, X_test_attention],workers = -1,callbacks=[tqdm_callback],verbose=1),axis = 1)
print(classification_report(y_test, y_pred,zero_division = 0))

461/461 [==============================] - 193s 418ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        73
           1       0.00      0.00      0.00       118
           2       0.00      0.00      0.00       538
           3       0.00      0.00      0.00       501
           4       0.00      0.00      0.00       586
           5       0.00      0.00      0.00      1719
           6       0.00      0.00      0.00       873
           7       0.00      0.00      0.00      1268
           8       0.36      0.27      0.31      2825
           9       0.00      0.00      0.00      1007
          10       0.00      0.00      0.00      1704
          11       0.00      0.00      0.00       722
          12       0.20      0.91      0.33      2790

    accuracy                           0.22     14724
   macro avg       0.04      0.09      0.05     14724
weighted avg       0.11      0.22      0.12     14724



In [14]:
# augmented EmoRoBERTa
roberta.load_weights('weights/augmented/saved_weights.h5')
y_pred = tf.argmax(roberta.predict([X_test_ids, X_test_attention],workers = -1,callbacks=[tqdm_callback],verbose=1),axis = 1)
print(classification_report(y_test, y_pred,zero_division = 0))

461/461 [==============================] - 192s 417ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        73
           1       0.00      0.00      0.00       118
           2       0.20      0.00      0.01       538
           3       0.00      0.00      0.00       501
           4       0.00      0.00      0.00       586
           5       0.34      0.35      0.35      1719
           6       0.42      0.58      0.49       873
           7       0.45      0.50      0.48      1268
           8       0.35      0.60      0.44      2825
           9       0.44      0.22      0.29      1007
          10       0.24      0.65      0.35      1704
          11       0.20      0.02      0.03       722
          12       0.43      0.04      0.07      2790

    accuracy                           0.33     14724
   macro avg       0.24      0.23      0.19     14724
weighted avg       0.33      0.33      0.27     14724

